In [ ]:
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Output, Input, State
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn import svm


df_original = sns.load_dataset("iris")

df = df_original.copy()

# codificando las variables nominales para el algoritmo

df["species_encoded"] = LabelEncoder().fit_transform(df["species"])

pca_model = PCA(n_components=2) 

pca = pca_model.fit_transform(df[df.columns[:4]])

df["PCA_1"] = pca.T[0]
df["PCA_2"] = pca.T[1]

df

#### PCA(Principal Component Analysis) 
técnica empleada basada en realizar ciertos pasos guiados para llegar a una dimensionalidad reducidad de datos. Sus ventajas y razones de uso son las siguientes: 

Reducción de dimensionalidad: como ya habíamos mencionado reduce la informacíon simplificando el problema en cuestión.

Visualización de datos: permite crear gráficos cartesianos que con tasa grandes de datos no podríamos.

Algoritmos simples: ayuda a algoritmos que utilizan métodos no tan complejos, como en este ejemplo.

In [ ]:
model = svm.SVC(kernel="rbf", C=1, probability=True)

model.fit(pca, df["species_encoded"])

object = [[df["sepal_length"].mean(), df["sepal_width"].mean(), df["petal_length"].mean(), df["petal_width"].mean()]]

pca_object = pca_model.transform(object)

# prediciendo la clase(especie) del nuevo objeto

predict_encoded = model.predict(pca_object)

# asociando las variables categóricas codificadas con sus versiones originales

classes = list(zip(df["species"].unique(),df["species_encoded"].unique()))

# asociando la predicción a su clase

predict_example = classes[predict_encoded[0]][0]

# obteniendo la probabilidad de que esa nueva variable sea de esa categoría

probability = model.predict_proba(pca_object)

probability = probability[0, predict_encoded]*100

probability = str(probability[0])

print("--------------------------------------------")
print("nueva especie \n")
for c in df.columns[:4]:
    print(f"{c}: {df[c].mean()}")
    
print(f"\npredicción: {predict_example} | probabilidad: {probability[:4]}%")
print("--------------------------------------------")

#### Dashboard que refleja la transformación de las varibles y permite solicitudes para generar predicciones


In [ ]:
app = dash.Dash(__name__)

graph_pca = go.Figure()
graph_pca.add_trace(go.Scatter(x=df["PCA_1"],y=df["PCA_2"],mode="markers",marker_color="blue",name="especies"))
graph_pca.update_layout(title="Figura PCA(principal components analysis)")
graph_pca.update_layout(legend=dict(font=dict(size=9)))

app.layout =  html.Div(id="body",className="e4_body",children=[
    html.H1("Iris",id="title",className="e4_title"),
    html.Div(id="dashboard",className="e4_dashboard",children=[
        html.Div(className="e4_graph_div",children=[
            dcc.Dropdown(id="dropdown",style={"font-famliy":"san-<sserif","width":"200px","height":"25px","margin":"3px 0 0 0"},
                        options=[
                            {"label":"tercer variable predictora","value":"petal_length"},
                            {"label":"cuarta variable predictora","value":"petal_width"}
                        ],
                        value="petal_length",
                        multi=False,
                        clearable=False),
            dcc.Graph(id="graph-1",className="e4_graph",figure={})
        ]),
        html.Div(className="e4_graph_div",children=[
            dcc.Graph(id="graph-2",className="e4_graph",figure=graph_pca),
            html.Form(id="input_div",className="input_div",children=[
                dcc.Input(id="input_1",className="input",type="text",placeholder="sepal_length",size="7"),
                dcc.Input(id="input_2",className="input",type="text",placeholder="sepal_width",size="7"),
                dcc.Input(id="input_3",className="input",type="text",placeholder="petal_length",size="7"),
                dcc.Input(id="input_4",className="input",type="text",placeholder="petal_width",size="7"),
                dcc.Input(id="button",className="input",type="button",value="enviar",n_clicks=0)
            ]),
            html.P(["predicción: ",html.B(children=[],id="predict")," | probabildad: ",html.B(children=[],id="probability"),"%"],className="e4_predict")
        ])
    ])
])
    
@app.callback(
    Output(component_id="graph-1",component_property="figure"),
    [Input(component_id="dropdown",component_property="value"),]
)

def update_graph(slct_var):
    
    graph_multi = px.scatter_3d(df,x='sepal_length',y='sepal_width',z=slct_var,color='species')
    graph_multi.update_layout(title="Figura multidimensional")
    graph_multi.update_layout(legend=dict(font=dict(size=9.5)))
    
    return graph_multi
    
@app.callback(
    [Output(component_id="graph-2",component_property="figure"),
    Output(component_id="predict",component_property="children"),
    Output(component_id="probability",component_property="children")],
    [Input(component_id="button",component_property="n_clicks")],
    [State(component_id="input_1",component_property="value"),
    State(component_id="input_2",component_property="value"),
    State(component_id="input_3",component_property="value"),
    State(component_id="input_4",component_property="value"),]
)

def update_graph(n_clicks, var_1, var_2, var_3, var_4):
    if n_clicks is not None and n_clicks > 0:
        object = [[var_1,var_2,var_3,var_4]]    
        pca_object = pca_model.transform(object)
        predict_encoded = model.predict(pca_object)
        predict = classes[predict_encoded[0]][0]
        probability = model.predict_proba(pca_object)
        probability = probability[0,predict_encoded]*100
        probability = str(probability[0])
        probability = probability[:4]

    graph_pca.add_trace(go.Scatter(x=[pca_object[0,0]],y=[pca_object[0,1]],mode="markers",marker_color="red",name=f"nueva especie({predict})"))
    
    return graph_pca, predict, probability

if __name__ == "__main__":
    app.run_server(debug=False)

#### Machine Learning Deployment
Hasta ahora hemos generado modelos predictivos que se encargan de resolver problemas asignados pero no fueron llevados a producción, es decir, tomar el modelo en particular desarrollado a partir de los fundamentos de Machine Learning, ponerlo a disposición de un usuario final y lograr que este modelo se pueda actualizar periódicamente para garantizar que tenga siempre el desempeño esperado. Para esto se requieren de habilidades menos científicas y más propiamente de un Ingenirero de software, donde se implementan herramientas como Docker que ayuda a crear y distribuir aplicaciones mediante contenedores que incluyan las librerías específicas instaladas, habiliten puertos, contengan volumes, etc y luego del uso de un orquestador como Kubernetes donde se gestionarán los recursos en los contenedores, servicios, configuraciones y la infraestructura en general del despliegue que se realizará. Como ya habíamos mencionado, esta área es un híbrido entre la ciencia de datos que abarca procesos como la extracción de datos, el análisis exploratorio, el preprocesamiento de datos, el entrenamiento de modelos y su evaluación y un ingeniero de softwre convencional, sin embargo, en el Machine Learning Deployment el software es mayormente dinámico, ya que el constante cambio en la información provoca que se requiera de fases posteirores al despliegue y puesta en servicio denominadas monitoreo y mantenimiento, el objetivo en estas fases es la verificación del desempeño del modelo y la repetición de las fases anteriormente mencionadas en el caso de que haya habido una degradación.